## Практическое задание к уроку 2 по теме "Создание признакового пространства".

Продолжим обработку данных с Твиттера. 

1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
    • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    • Исключим стоп-слова с помощью stop_words='english'.
    • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().
  
2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.
    • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
    • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
    • Исключим стоп-слова с помощью stop_words='english'.
    • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().  
3. Натренируем gensim.models.Word2Vec модель на наших данных.
    • Тренировать будем на токенизированных твитах combine_df['tweet_token']
    • Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34.
    • Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.
  
4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".  
  
5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).
Давайте проверим векторное представление любого слова из нашего корпуса, например "food".  

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.
Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size))
и поделить финальный вектор на количество слов в твите.
На выходе должен получиться wordvec_df.shape = (49159, 200).

Загрузим данные с предыдущего практического задания:

In [1]:
import pandas as pd
import pickle

In [2]:
with open('../Lesson_1/data/preprocessed_data.pkl', 'rb') as f:
    df = pickle.load(f)

In [3]:
df.head()

,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
id,,,,,,
1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


1.
Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно. • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df. • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000. • Исключим стоп-слова с помощью stop_words='english'. • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df['tweet_stemmed'] = df['tweet_stemmed'].apply(lambda x: ' '.join(x))
df['tweet_lemmatized'] = df['tweet_lemmatized'].apply(lambda x: ' '.join(x))

In [6]:
cvec_stem = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')
cvec_lemm = CountVectorizer(max_df=0.9, max_features=1000, stop_words='english')

bow_cv_stem = cvec_stem.fit_transform(df['tweet_stemmed'])
bow_cv_lemm = cvec_lemm.fit_transform(df['tweet_lemmatized'])

In [7]:
bow_cv_stem = pd.DataFrame.sparse.from_spmatrix(bow_cv_stem, columns=cvec_stem.get_feature_names_out())
bow_cv_lemm = pd.DataFrame.sparse.from_spmatrix(bow_cv_lemm, columns=cvec_lemm.get_feature_names_out())

In [8]:
bow_cv_stem.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно. • Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df. • Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000. • Исключим стоп-слова с помощью stop_words='english'. • Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tvec_stem = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')
tvec_lemm = TfidfVectorizer(max_df=0.9, max_features=1000, stop_words='english')

bow_tv_stem = tvec_stem.fit_transform(df['tweet_stemmed'])
bow_tv_lemm = tvec_lemm.fit_transform(df['tweet_lemmatized'])

In [11]:
bow_tv_stem = pd.DataFrame.sparse.from_spmatrix(bow_tv_stem, columns=tvec_stem.get_feature_names_out())
bow_tv_lemm = pd.DataFrame.sparse.from_spmatrix(bow_tv_lemm, columns=tvec_lemm.get_feature_names_out())

In [12]:
bow_tv_lemm.head()

,able,absolutely,account,act,action,actor,actually,adapt,add,adventure,...,year,yes,yesterday,yo,yoga,york,young,youtube,yr,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


3. Натренируем gensim.models.Word2Vec модель на наших данных. • Тренировать будем на токенизированных твитах combine_df['tweet_token'] • Установим следующие параметры: size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34. • Используем функцию train() с параметром total_examples равным длине combine_df['tweet_token'], количество epochs установим 20.

In [13]:
from gensim.models import word2vec

In [14]:
w2v = word2vec.Word2Vec(df['tweet_token'], vector_size=200, window=5, min_count=2, sg=1, hs=0, negative=10, seed=34, workers=12)
w2v.train(df['tweet_token'], total_examples=len(df), epochs=20)

(9199294, 11790760)

4. Давайте немного потестируем нашу модель Word2Vec и посмотрим, как она работает. Мы зададим слово positive = "dinner", и модель вытащит из корпуса наиболее похожие слова c помощью функции most_similar. То же самое попробуем со словом "trump".

In [15]:
w2v.wv.most_similar(positive='dinner')

[('bihdaydinner', 0.5568002462387085),
 ('cookout', 0.5498145818710327),
 ('bolognese', 0.5429084300994873),
 ('shawarma', 0.533199667930603),
 ('tacotuesday', 0.5323346257209778),
 ('hamburger', 0.5248205661773682),
 ('spaghetti', 0.522797167301178),
 ('waterloo', 0.5203596353530884),
 ('lastnight', 0.5141153931617737),
 ('lamb', 0.5126266479492188)]

In [16]:
w2v.wv.most_similar(positive='trump')

[('donald', 0.553157389163971),
 ('suppoer', 0.5381174683570862),
 ('trumptrain', 0.518347978591919),
 ('unfavorability', 0.5142640471458435),
 ('dumptrump', 0.5138635039329529),
 ('impeachment', 0.5101327896118164),
 ('donaldtrump', 0.5077646374702454),
 ('landslide', 0.5061473846435547),
 ('unstable', 0.5021856427192688),
 ('racists', 0.5021058320999146)]

5. Из приведенных выше примеров мы видим, что наша модель word2vec хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова). Давайте проверим векторное представление любого слова из нашего корпуса, например "food".

In [17]:
w2v.wv['food']

array([ 0.47018722,  0.19807665,  0.51361686, -0.29027238, -0.08123974,
       -0.15692584, -0.08659583, -0.07750746, -0.4769662 ,  0.07051579,
        0.1553299 ,  0.664919  , -0.22626036,  0.432069  ,  0.41472256,
       -0.46789902,  0.7674717 , -0.06155152, -0.19858138, -0.6964467 ,
       -0.31180942,  0.02801759, -0.9658839 ,  0.0043061 ,  0.04740566,
        0.7526299 , -1.2153084 , -0.4102754 ,  0.27914888, -0.19093771,
        0.3142806 , -0.21757497, -0.15684263,  0.20230335, -1.1284038 ,
       -0.3630038 ,  0.8751802 , -0.0635411 ,  0.14203243, -0.6639904 ,
       -0.06075408, -0.45891157,  0.08201483,  0.7440632 ,  0.23648736,
       -0.08807326, -0.4614497 , -0.4500064 , -0.37902114, -0.6233122 ,
        0.04432596, -0.06405585,  0.02395995, -0.26261368, -0.3487461 ,
       -0.9373858 , -0.2622752 ,  0.03379533, -0.34526038,  0.5597952 ,
       -0.36933407,  0.16252871,  0.04648735, -0.44571123, -0.23196456,
        0.212826  ,  0.901017  ,  0.66262233,  0.15821247,  0.21

6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита. Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных. Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать: vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите. На выходе должен получиться wordvec_df.shape = (49159, 200).

In [18]:
import numpy as np

In [19]:
def get_sentence_vec(sentence, model):
    sum_vec = np.zeros((1, model.vector_size))
    i = 0
    
    for word in sentence['tweet_token']:
        if word in model.wv:
            sum_vec += model.wv[word].reshape(1, -1)
            i += 1
            
    return np.squeeze(sum_vec / i) if i else np.squeeze(sum_vec)

In [20]:
tweet_vectors = df.apply(get_sentence_vec, axis=1, result_type='expand', model=w2v)

In [21]:
tweet_vectors.head(3)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
id,,,,,,,,,,,,,,,,,,,,,
1,-0.029095,-0.120969,0.231957,-0.091578,-0.165193,0.020818,-0.205122,-0.267923,0.076370,0.009890,...,0.177879,-0.286595,0.073362,-0.089823,0.181927,0.201729,0.000677,-0.221264,0.021728,-0.008503
2,-0.199682,-0.271347,0.055071,-0.146986,-0.233806,-0.091754,-0.227607,-0.293015,-0.026972,-0.085604,...,0.051128,-0.307307,0.171786,-0.079836,0.073574,0.165274,0.006235,-0.212228,0.189741,0.005495
3,-0.514459,-0.079656,0.091082,0.191382,-0.215843,0.155935,-0.257496,-0.469222,0.112373,-0.193821,...,0.204260,-0.758033,0.073523,-0.241930,0.012007,0.216654,-0.030310,0.102259,0.407475,0.385974


In [22]:
tweet_vectors.shape

(49159, 200)